# Traffic Light Classifier using Bosch dataset
I’ll be creating a traffic light classifier which will try to determine if the light is green, yellow, or red. Currently the pre-trained models only try to detect if there is a traffic light in the image, not the state of the traffic light.

Steps:
1. selecting a model
2. adapting an existing data set
3. creating and annotating your own data set
4. modifying the model config file
5. training the model
6. saving the model
7. finally deploying the model in another piece of software.

#### Installation

Download the repos

In [0]:
!git clone https://github.com/tensorflow/models.git 

Cloning into 'models'...
remote: Enumerating objects: 24307, done.
remote: Total 24307 (delta 0), reused 0 (delta 0), pack-reused 24307
Receiving objects: 100% (24307/24307), 507.59 MiB | 30.36 MiB/s, done.
Resolving deltas: 100% (14131/14131), done.
Checking out files: 100% (2808/2808), done.


In [1]:
!git clone https://github.com/ThomasHenckel/CarND-Capstone

Cloning into 'CarND-Capstone'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 726 (delta 8), reused 18 (delta 8), pack-reused 699
Receiving objects: 100% (726/726), 41.25 MiB | 28.97 MiB/s, done.
Resolving deltas: 100% (417/417), done.


Download Bosch dataset

In [0]:
!mkdir dataset && cd dataset

In [0]:
!wget https://www.kaggle.com/mbornoe/lisa-traffic-light-dataset/downloads/lisa-traffic-light-dataset.zip/2
 

Download LISA dataset (from Kaggle)

In [13]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ilopezfr","key":"54a1d43df05cc80ab9d46edb4b102252"}'}

In [14]:
# Setup kaggle Dir; copy json file; chmod
import os
os.chdir('/content/dataset') 
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [15]:
# verify json content
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

total 4
-rw------- 1 root root 64 Mar  3 00:39 kaggle.json
{"username":"ilopezfr","key":"54a1d43df05cc80ab9d46edb4b102252"}

In [16]:
#Install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

#List Kaggle DataSets
!kaggle datasets list

    100% |████████████████████████████████| 81kB 5.5MB/s 
    100% |████████████████████████████████| 5.3MB 1.1MB/s 
    100% |████████████████████████████████| 92kB 24.0MB/s 
    100% |████████████████████████████████| 51kB 19.8MB/s 
ref                                                          title                                                size  lastUpdated          downloadCount  
-----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
ronitf/heart-disease-uci                                     Heart Disease UCI                                     3KB  2018-06-25 11:33:56          11533  
russellyates88/suicide-rates-overview-1985-to-2016           Suicide Rates Overview 1985 to 2016                 396KB  2018-12-01 19:18:25           8074  
karangadiya/fifa19                                           FIFA 19 complete player dataset                       2MB  2018-12-21 03:52:

In [19]:
import os
os.chdir('/content/dataset')
!kaggle datasets download -d mbornoe/lisa-traffic-light-dataset

lisa-traffic-light-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [20]:
os.chdir('/content/dataset')
!unzip lisa-traffic-light-dataset.zip

Archive:  lisa-traffic-light-dataset.zip
  inflating: daySequence1.zip        
  inflating: nightSequence1.zip      
  inflating: dayTrain.zip            
  inflating: daySequence2.zip        
  inflating: sample-dayClip6.zip     
  inflating: sample-nightClip1.zip   
  inflating: nightTrain.zip          
  inflating: Annotations.zip         
  inflating: nightSequence2.zip      


In [0]:
os.chdir('/content/dataset')
!unzip daySequence1.zip && unzip Annotations.zip

Download Udacity Camera Feed

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [36]:

!unzip "/content/gdrive/My Drive/Colab Notebooks/data/traffic_light_bag_files.zip" -d /content/dataset

Archive:  /content/gdrive/My Drive/Colab Notebooks/data/traffic_light_bag_files.zip
replace /content/dataset/just_traffic_light.bag? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/dataset/just_traffic_light.bag  y

  inflating: /content/dataset/loop_with_traffic_light.bag  A



In [0]:
# Move traffic_light_dataset to the datasets location:
import shutil
shutil.move("/content/gdrive/My Drive/Colab Notebooks/data/traffic_light_dataset", "/content/dataset")

In [47]:
!unzip /content/dataset/traffic_light_dataset/carla/carla.zip

Archive:  /content/dataset/traffic_light_dataset/carla/carla.zip
replace carla/testing/labels/uda_2_g_0001.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [50]:
import os
os.chdir('/content/models/research')

os.environ['PYTHONPATH'] += ":/content/models"
os.getcwd()

'/content/models/research'

In [0]:
#!pip3 install --user -r models/official/requirements.txt
!pip3 install --user Cython
!pip3 install --user contextlib2
!pip3 install --user jupyter
!pip3 install --user matplotlib

In [54]:
# COCO API Installation ---> not working
# From tensorflow/models/research/
os.chdir('/content/models/research')
!rm -r cocoapi
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 947, done.
remote: Total 947 (delta 0), reused 0 (delta 0), pack-reused 947
Receiving objects: 100% (947/947), 11.69 MiB | 36.15 MiB/s, done.
Resolving deltas: 100% (565/565), done.


In [59]:
os.chdir('/content/models/research/cocoapi/PythonAPI')
!bash -c 'make' 
#!sudo make install
#!bash -c 'cp -r content/models/research/cocoapi/PythonAPI/pycocotools content/models/research'

python setup.py build_ext --inplace
Traceback (most recent call last):
  File "setup.py", line 1, in <module>
    from setuptools import setup, Extension
  File "/usr/local/lib/python3.6/dist-packages/setuptools/__init__.py", line 20, in <module>
    from setuptools.dist import Distribution, Feature
  File "/usr/local/lib/python3.6/dist-packages/setuptools/dist.py", line 36, in <module>
    from setuptools import windows_support
  File "/usr/local/lib/python3.6/dist-packages/setuptools/windows_support.py", line 2, in <module>
    import ctypes
  File "<frozen importlib._bootstrap>", line 968, in _find_and_load
  File "<frozen importlib._bootstrap>", line 149, in __enter__
  File "<frozen importlib._bootstrap>", line 88, in acquire
KeyboardInterrupt
Makefile:3: recipe for target 'all' failed
make: *** [all] Error 1
^C


In [0]:
# Protobuf compilation
# From tensorflow/models/research/
os.chdir('/content/models/research')
#!protoc object_detection/protos/*.proto --python_out=.
!wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
!unzip protobuf.zip
!./bin/protoc object_detection/protos/*.proto --python_out=.


In [0]:
# From tensorflow/models/research/
#export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
os.chdir('/content/models/research')
os.environ['PYTHONPATH'] += ":/content/models"

In [64]:
# Test the installation
os.chdir('/content/models/research')
!python object_detection/builders/model_builder_test.py

Traceback (most recent call last):
  File "object_detection/builders/model_builder_test.py", line 23, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


# Training a Model




In [74]:
# move object detection models from TF to our CarND-Capstone folder
os.chdir('/content/CarND-Capstone/Traffic_Light_Detection')
!mkdir models

#!cp -r /content/models/research/object_detection content/CarND-Capstone/Traffic_Light_Detection
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz

--2019-03-03 05:44:46--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128048406 (122M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_coco_11_06_2017.tar.gz.2’

ssd_mobilenet_v1_co 100%[===================>] 122.12M   214MB/s    in 0.6s    

2019-03-03 05:44:46 (214 MB/s) - ‘ssd_mobilenet_v1_coco_11_06_2017.tar.gz.2’ saved [128048406/128048406]



In [76]:
!tar -xvzf  /content/CarND-Capstone/Traffic_Light_Detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz -C /content/CarND-Capstone/Traffic_Light_Detection/models

ssd_mobilenet_v1_coco_11_06_2017/
ssd_mobilenet_v1_coco_11_06_2017/model.ckpt.index
ssd_mobilenet_v1_coco_11_06_2017/model.ckpt.meta
ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb
ssd_mobilenet_v1_coco_11_06_2017/model.ckpt.data-00000-of-00001
ssd_mobilenet_v1_coco_11_06_2017/graph.pbtxt


In [0]:
#Run the training session
python object_detection/model_main.py --pipeline_config_path=content/CarND-Capstone/Traffic_Light_Detection/training/ssd_mobilenet_v1_coco.config --model_dir=models/fine_tuned/ssd_inception


# Testing a Model: Object Detection Demo

In the models/research/objection_detection/ folder, open up the jupyter notebook object_detection_tutorial.ipynb and run the entire notebook. 
(https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb)


## Imports

In [0]:
# Make sure you are in /content/models/research. If not, use:
# os.chdir('/content/models/research')
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import static_shape
from object_detection.utils import shape_utils
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [0]:
# Env Setup - This is needed to display the images.
%matplotlib inline

os.chdir('/content/models/research')

# Object detection imports
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


# Model Preparation

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.


In [0]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download the model

In [0]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label_maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
label_map = label_map_util.load_labelmap(os.path.join('/home/user/anaconda3/lib/python3.6/site-packages/tensorflow/models/research/object_detection/data', 'mscoco_label_map.pbtxt'))

categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


NotFoundError: ignored

In [0]:

from object_detection.utils import label_map_util

#category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
label_map = label_map_util.load_labelmap(os.path.join('anaconda3/lib/python3.6/site-packages/tensorflow/models/research/object_detection/data', 'mscoco_label_map.pbtxt'))
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=4, use_display_name=True) # Loads label map proto and returns categories list compatible with eval.
category_index = label_map_util.create_category_index(categories) # Creates dictionary of COCO compatible categories keyed by category id.
print(category_index)

NotFoundError: ignored